In [7]:
import json
import xml.etree.ElementTree as ET
import os

xml_fp = "data/pp_all_viac_securities.xml"
viac_funds_fp = "data/viac_funds.json"


# Open and read the JSON file, this json file is extracted from the viac website in the custom strategy interface
with open(viac_funds_fp, 'r') as file:
    data = json.load(file)

# Extract the list of securities
securities = data.get('securities', [])

# Create a list to store the required information
securities_dict = {}

# Iterate through each security and extract the required fields
for security in securities:
    isin = security.get('isin')
    name = security.get('name')
    currency_code = security.get('currencyCode')
    securities_dict[name] = (isin, currency_code)
    print(isin, currency_code, name)



CH0033846350 CHF UBS Bond CH AAA-AA
CH0281860111 CHF UBS Bond CH Corporate
CH0033210086 CHF UBS Bond World ex CH Gov.
CH0189955260 CHF UBS Bond World ex CH Corp.
CH0259132105 USD UBS Bond Emerging Markets
CH0033782431 CHF UBS SMI
CH0110869143 CHF UBS SPI Extra
CH0032912732 CHF UBS ETF SLI
IE00B5BMR087 USD iShares Core S&P500
LU0629460089 USD UBS ETF MSCI USA SRI
CH0030849613 CAD UBS Canada
LU1048313891 USD UBS ETF MSCI Emerging Markets SRI
LU0629460675 EUR UBS ETF MSCI EMU SRI
IE00BMP3HN93 GBP UBS ETF MSCI UK IMI SRI
LU0629460832 USD UBS ETF MSCI Pacific SRI
CH0036599816 CHF UBS Real Estate CH
IE00B1FZSF77 USD iShares US Property Yield
CH0209106761 USD CSIF Gold
IE00B1TXHL60 USD iShares Listed Private Equity
LU0340004091 USD UBS Global High Yield Bond Fund
CH0368190739 CHF UBS ETF MSCI Switzerland IMI SRI
LU0439730705 USD UBS Global Quality Dividend
IE00B53SZB19 USD iShares Nasdaq 100
CH0397628709 USD CSIF US ESG - Pension Fund
CH0030849712 USD CSIF US - Pension Fund
CH0214967314 CHF C

In [11]:
# Parse the XML file
tree = ET.parse(xml_fp)
root = tree.getroot()

# double check consistency between project xml and the dict above
for security in root.findall('.//security'):
    isin = security.find('isin').text if security.find('isin') is not None else None
    name = security.find('name').text if security.find('name') is not None else None
    currency_code = security.find('currencyCode').text if security.find('currencyCode') is not None else None
    
    if name in securities_dict:
        isin_target = securities_dict[name][0]
        currency_target = securities_dict[name][1]
        if currency_code != currency_target:
            print("{} currency should be {} but is {}".format(name, currency_target, currency_code))
    else:
        print("Fund is not in the current VIAC line-up anymore and not tested for consistency: {}".format(name))
    

Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Europe ex CH Real Estate
Fund is not in the current VIAC line-up anymore and not tested for consistency: UBS Europe ex CH (old)
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Asia Real Estate
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Japan
Fund is not in the current VIAC line-up anymore and not tested for consistency: UBS Pacific ex Japan (old)
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF World ex CH - Pension Fund
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Real Estate World ex CH - Pension Fund (old)
Fund is not in the current VIAC line-up anymore and not tested for consistency: UBS Emerging Markets (old)


In [12]:
# Parse the XML file
tree = ET.parse(xml_fp)
root = tree.getroot()

# Iterate through each security in the XML
for security in root.findall('.//security'):
    isin = security.find('isin').text if security.find('isin') is not None else None
    name = security.find('name').text if security.find('name') is not None else None
    currency_code = security.find('currencyCode').text if security.find('currencyCode') is not None else None
    
    if name in securities_dict:
        isin_target = securities_dict[name][0]
        currency_target = securities_dict[name][1]

        if isin is not None:
            if isin != isin_target:
                print(f"Error: ISIN value {isin} for {name} does not match target {isin_target}")
        else:
            # Add ISIN tag with the target value
            new_isin = ET.SubElement(security, 'isin')
            new_isin.text = isin_target

        if currency_code != currency_target:
            print(f"{name} currency should be {currency_target} but is {currency_code}")
    else:
        print("Fund is not in the current VIAC line-up anymore and not tested for consistency: {}".format(name))

# Save the modified XML to a file
tree = ET.ElementTree(root)
tree.write(xml_fp, encoding='unicode')

print("Modified XML has been saved to {}".format(xml_fp))

Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Europe ex CH Real Estate
Fund is not in the current VIAC line-up anymore and not tested for consistency: UBS Europe ex CH (old)
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Asia Real Estate
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Japan
Fund is not in the current VIAC line-up anymore and not tested for consistency: UBS Pacific ex Japan (old)
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF World ex CH - Pension Fund
Fund is not in the current VIAC line-up anymore and not tested for consistency: CSIF Real Estate World ex CH - Pension Fund (old)
Fund is not in the current VIAC line-up anymore and not tested for consistency: UBS Emerging Markets (old)
Modified XML has been saved to data/pp_all_viac_securities.xml
